In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/saeta/Just", from: "0.7.2")' Just
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/saeta/Just", from: "0.7.2")
		Just
	.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")
		NotebookExport
With SwiftPM flags: []
Working in: /tmp/tmpyhs5suvt/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [2]:
// export

// Source: The original python implementation can be found here:
// https://github.com/mortendahl/privateml/blob/master/image-analysis/pond/tensor.py

import TensorFlow

let Q:Int64 = 12332434434

let BASE = 3
let PRECISION_INTEGRAL   = 6
let PRECISION_FRACTIONAL = 5
let SCALING_FACTOR = pow(Double(BASE), Double(PRECISION_FRACTIONAL))


func mod(_ x: Tensor<Int64>, _ q: Int64) -> Tensor<Int64>{
  return Raw.floorMod(x, Tensor<Int64>([q]))
}

extension TensorShape {
  func countElements() -> Int{
    var nb_el = 1
    for i in 0 ..< self.count{
      nb_el = nb_el * self[i]
    }
    return nb_el
  }
}

public func _encode(rationals: Tensor<Double>) -> Tensor<Int64>{
    let scaled = rationals * SCALING_FACTOR
    let integer = mod(Tensor<Int64>(scaled), Q)
    return integer
}

func _decode(field_element: Tensor<Int64>) -> Tensor<Double>{
      let gr_q = Tensor<Int64>(_Raw.greaterEqual(field_element, Tensor<Int64>([Q/2]))) // 1 if val > Q/2 else 0
      let scaled = field_element - gr_q * Q
      let scaled_rationals = Tensor<Double>(scaled)
      return scaled_rationals / SCALING_FACTOR
}

// NOTE Should be able to use Tensor(randomUniform instead)
func makeRandomList(_ n: Int, _ q: Int64) -> [Int64] {
    return (0..<n).map { _ in Int64.random(in: 0..<q) }
}

func _share(x: Tensor<Int64>) ->  (Tensor<Int64>, Tensor<Int64>){
    let share0 = Tensor<Int64>(makeRandomList(x.shape.countElements(), Q)).reshaped(to: x.shape)
    let share1 = mod(x - share0, Q)
    return (share0, share1)
}

func reconstruct(share0: Tensor<Int64>, share1: Tensor<Int64>) -> Tensor<Int64>{
  let rec = mod(share0 + share1, Q)
  return rec
}

In [3]:
let x: Tensor<Double> = Tensor([[1.0, 2.0], [3.0, 4.0]])
let x_enc = _encode(rationals: x)
let x_shared = _share(x: x_enc)
let x_rec = _decode(field_element: reconstruct(share0: x_shared.0, share1: x_shared.1))
print(x_rec)

[[1.0, 2.0],
 [3.0, 4.0]]


In [4]:
// export
enum TensorError: Error{
  case tensorType(String)
}

func errorTypeMessage<T, U>(_ x: T, _ y: U) -> String {
  return "\(type(of: x)) doesn't support  \(type(of: y))"
}

public protocol PondTensor {}

// PublicTensor

public struct PublicTensor: PondTensor {
  let elements: Tensor<Int64>

  init(elements: Tensor<Int64>){
    self.elements = elements
  }

  public static func from_values(values: Tensor<Double>) -> PublicTensor{
    let enc_val = _encode(rationals: values)

    return PublicTensor(elements: enc_val)
  }

  public func unwrap() -> Tensor<Int64>{
    return self.elements
  }

  public func reveal() -> PublicTensor{
    return self
  }

  public func decode() -> Tensor<Double>{
    return _decode(field_element: self.elements)
  }

  func add(y: PublicTensor) -> PublicTensor{
    let elements = mod(self.elements + y.elements, Q)
    return PublicTensor(elements: elements)
  }
    
  func mul(y: PublicTensor) -> PublicTensor{
    let elements = mod(_Raw.mul(self.elements, y.elements), Q)
    return PublicTensor(elements: elements).truncate()
  }
    

  public func matmul(y: PublicTensor) -> PublicTensor{
    let elements = mod(_Raw.matMul(self.elements, y.elements), Q)
    return PublicTensor(elements: elements).truncate()
  }
 
    
  public func truncate() -> PublicTensor {
    let positive_numbers = Tensor<Int64>(_Raw.lessEqual(self.elements, Tensor<Int64>([Q/2])))
    var elements = self.elements
    elements = mod((Q + (2 * positive_numbers - 1) * elements), Q)
    elements = _Raw.floorDiv(elements, Tensor<Int64>([Int64(SCALING_FACTOR)]))
    elements = mod((Q + (2 * positive_numbers - 1) * elements), Q) //#

    return PublicTensor(elements: elements)
   }
 }


// PublicFieldTensor

public struct PublicFieldTensor: PondTensor {
  let elements: Tensor<Int64>

  init(elements: Tensor<Int64>){
    self.elements = elements
  }

  public static func from_values(values: Tensor<Int64>) -> PublicFieldTensor{
    let enc_val = values
    return PublicFieldTensor(elements: enc_val)
  }

  public func reveal() -> PublicFieldTensor {
    return self
  }
    
  public func add(y: PublicFieldTensor) -> PublicFieldTensor{
    let elements = mod(self.elements + y.elements, Q)
    return PublicFieldTensor(elements: elements)
  }

  public func add(y: PrivateFieldTensor) -> PrivateFieldTensor{
    let share0 = mod(self.elements + y.share0, Q)
    let share1 = y.share1
    return PrivateFieldTensor(share0: share0, share1: share1)
  }

  func matmul(y: PublicFieldTensor) -> PublicFieldTensor{
      let elements = Raw.matMul(self.elements, y.elements) % Q
      return PublicFieldTensor(elements: elements)
  }

  func matmul(y: PrivateFieldTensor) -> PrivateFieldTensor{
      let share0 = mod(Raw.matMul(self.elements, y.share0), Q)
      let share1 = mod(Raw.matMul(self.elements, y.share1), Q)
      return PrivateFieldTensor(share0: share0, share1: share1)
    }
}


// PrivateFielTensor 

public struct PrivateFieldTensor: PondTensor{
  let share0: Tensor<Int64>
  let share1: Tensor<Int64>

  init(share0: Tensor<Int64>, share1: Tensor<Int64>){
    self.share0 = share0
    self.share1 = share1
  }

  public static func from_values(values: Tensor<Int64>) -> PrivateFieldTensor{
    let x_shared = _share(x: values)
    return PrivateFieldTensor(share0: x_shared.0, share1: x_shared.1)
  }

  public func reveal() -> PublicFieldTensor{
  let x_rec = reconstruct(share0: self.share0, share1: self.share1)
    return PublicFieldTensor.from_values(values: x_rec)
  }
    
  public func add(y: PrivateFieldTensor) -> PrivateFieldTensor{
    let share0 = mod(self.share0 + y.share0, Q)
    let share1 = mod(self.share1 + y.share1, Q)
    return PrivateFieldTensor(share0: share0, share1: share1)
  }
    
 public func add(y: PublicFieldTensor) -> PrivateFieldTensor{
    let share0 = mod(self.share0 + y.elements, Q)
    let share1 = mod(self.share1, Q)
    return PrivateFieldTensor(share0: share0, share1: share1)
  }

public func matmul(y: PublicFieldTensor) -> PrivateFieldTensor{
    let share0 = Raw.matMul(self.share0, y.elements) % Q
    let share1 = Raw.matMul(self.share1, y.elements) % Q
    return PrivateFieldTensor(share0: share0, share1: share1)
    }
}

In [5]:
// export
func generate_dot_triple(m: Int, n: Int, o: Int) -> (PrivateFieldTensor, 
                                                     PrivateFieldTensor, 
                                                     PrivateFieldTensor){

    let a = Tensor<Int64>(makeRandomList(m*n, Q)).reshaped(to: [m, n])
    let b = Tensor<Int64>(makeRandomList(n*o, Q)).reshaped(to: [n, o])
    let ab = Raw.matMul(a, b)
    return (PrivateFieldTensor.from_values(values: Tensor<Int64>(a)), 
            PrivateFieldTensor.from_values(values: Tensor<Int64>(b)), 
            PrivateFieldTensor.from_values(values: Tensor<Int64>(ab)))
}

In [6]:
// export
public struct PrivateTensor: PondTensor {
  let share0: Tensor<Int64>
  let share1: Tensor<Int64>

  var shape: TensorShape {
        return share0.shape
    }

  init(share0: Tensor<Int64>, share1: Tensor<Int64>){
    self.share0 = share0
    self.share1 = share1
  }

  public static func from_values(values: Tensor<Double>) -> PrivateTensor{
    let x_shared = _share(x: _encode(rationals: values))
    return PrivateTensor(share0: x_shared.0, share1: x_shared.1)
  }

  public func reveal() -> PublicTensor{
    let x_rec = _decode(field_element: reconstruct(share0: self.share0, share1: self.share1))
    return PublicTensor.from_values(values: x_rec)
  }

  public func truncate() -> PrivateTensor {
    let share0 = _Raw.floorDiv(self.share0, Tensor<Int64>([Int64(SCALING_FACTOR)]))
    let share1 =  (Q - _Raw.floorDiv(Q - self.share1, Tensor<Int64>([Int64(SCALING_FACTOR)]))) % Q
    return PrivateTensor(share0: share0, share1: share1)
   }
    
    
  func add(y: PublicTensor) -> PrivateTensor{
    let share0 =  mod(self.share0 + y.elements, Q)
    let share1 = self.share1
    return PrivateTensor(share0: share0, share1: share1)
  }
    
  func add(y: PrivateTensor) -> PrivateTensor{
    let share0 = mod(self.share0 + y.share0, Q)
    let share1 = mod(self.share1 + y.share1, Q)
    return PrivateTensor(share0: share0, share1: share1)
  }
    
  func sub(y: PublicTensor) -> PrivateTensor{
    let share0 = mod(self.share0 - y.elements, Q)
    let share1 = self.share1
    return PrivateTensor(share0: share0, share1: share1)
  }
    
  func sub(y: PrivateTensor) -> PrivateTensor{
    let share0 = mod(self.share0 - y.share0, Q)
    let share1 = mod(self.share1 - y.share1, Q)
    return PrivateTensor(share0: share0, share1: share1)
  }

  public func sub(y: PrivateFieldTensor) -> PrivateFieldTensor{
      let share0 = mod(self.share0 - y.share0, Q)
      let share1 = mod(self.share1 - y.share1, Q)
      return PrivateFieldTensor(share0: share0, share1: share1)
      }


  public func matmul(y: PrivateTensor) throws -> PrivateTensor{
    let m = self.shape[0]
    let n = self.shape[1]
    let o = y.shape[1]

    let triple = generate_dot_triple(m: m, n: n, o: o)
    let a = triple.0 
    let b = triple.1 
    let ab = triple.2
      
    let alpha_r = self.sub(y: a).reveal()
    let beta_r = y.sub(y: b).reveal()
    let z_1 = alpha_r.matmul(y: beta_r)
    let z_2 = alpha_r.matmul(y: b)
    let z_3 = a.matmul(y: beta_r)
    let z = z_1.add(y: z_2).add(y: z_3).add(y: ab)
      
    return PrivateTensor(share0: z.share0 , share1: z.share1).truncate()
  }

}

In [7]:
let x = PrivateTensor.from_values(values: Tensor([[1, 2], [3, 4]]))
let b = PrivateTensor.from_values(values: Tensor([[1, 2], [3, 4]]))
let c = PrivateFieldTensor.from_values(values: Tensor([[1, 2], [3, 4]]))
let res_add = x.add(y: b)
let res_sub = x.sub(y: b)


print(res_add.reveal().decode())
print(res_sub.reveal().decode())

[[2.0, 4.0],
 [6.0, 8.0]]
[[0.0, 0.0],
 [0.0, 0.0]]


In [8]:
import NotebookExport
import Path
let exporter = NotebookExport(Path.cwd/"tensor.ipynb")
print(exporter.export(usingPrefix: "ppmlNB_"))

success
